Imports

In [61]:
from PIL import Image
import numpy as np
import cv2
from pylfsr import LFSR

## Helper Functions

- Any type to Binary Converter


In [62]:
def convertToBinary(data):
    if isinstance(data, str):
        return ''.join([ format(ord(i), "08b") for i in data ])
    elif isinstance(data, bytes) or isinstance(data, np.ndarray):
        return [ format(i, "08b") for i in data ]
    elif isinstance(data, int) or isinstance(data, np.uint8):
        return format(data, "08b")
    else:
        raise TypeError("Type not supported.")

In [63]:
print(list(convertToBinary(256)))

['1', '0', '0', '0', '0', '0', '0', '0', '0']


- Binary to Decimal Converter

In [64]:
def binaryToDecimal(binary):
    decimal = 0
    for bit in binary:
        decimal = decimal*2 + int(bit)
    return decimal

- Convert image to matrix

In [65]:
def getImageMatrix(imageName):

    imageHandler = Image.open(imageName)
    pixels = imageHandler.load()
    color = 1
    if type(pixels[0,0]) == int:
        color = 0
    image_size = imageHandler.size
    
    rows = int(image_size[0])
    cols = int(image_size[1])
    image_matrix = []
    for row in range(rows):
        current_row = []
        for col in range(cols):
            current_row.append(pixels[row,col])
        image_matrix.append(current_row)
    
    return image_matrix, image_size, color


- Extract Frames From Video

In [66]:
def FrameCapture(path): 
  
    vidObj = cv2.VideoCapture(path) 
    count = 0
  
    success = 1
  
    while count < 100: 
        success, image = vidObj.read()
        if not success:
            print("Video End")
            return  
        count += 1
    
    cv2.imwrite("images/frame1.jpg", image) 

    while count < 200: 
        success, image = vidObj.read() 
        if not success:
            print("Video End")
            return 
        count += 1

    cv2.imwrite("images/frame2.jpg", image) 

    while count < 300: 
        success, image = vidObj.read() 
        if not success:
            print("Video End")
            return 
        count += 1

    cv2.imwrite("images/frame3.jpg", image) 

    while count < 400: 
        success, image = vidObj.read() 
        if not success:
            print("Video End")
            return 
        count += 1

    cv2.imwrite("images/frame4.jpg", image) 


    

In [67]:
FrameCapture("images/original_video.mp4")

Video End


## 2D Chaotic Map

- Hennon Map (Binary)

In [68]:
def HennonMapBinary(dimension, key):

    rows = dimension[0]
    cols = dimension[1]
    x = key[0]
    y = key[1]
    a = 1.4
    b = 0.3

    # Total Number of bitSequence produced
    sequenceSize = rows * cols
    bitSequenceSize = cols
    # Each bitSequence contains k bits
    bitSequence = []       
    # Each matrix contains m*n byteArray
    matrix = []

    for i in range(sequenceSize):
        x_next = y + 1 - (a * (x**2)) 
        y_next = b * x
        x = x_next
        y = y_next
        if x <= 0.4:
            bit = 0
        else:
            bit = 1

        bitSequence.append(bit) 

        # If Bit Sequence has k bits, convert it to decimal and add it to Byte Sequence.
        if i % bitSequenceSize == bitSequenceSize - 1:
            matrix.append(bitSequence)
            bitSequence = []

    return matrix

## Encoding


In [72]:
def decode(videoname, hennonKey, poly, data_len):

    # Divide Encoded video into frames
    vidObj = cv2.VideoCapture("images/encoded_video.mp4") 
    success = True
    frames = []
    while success: 
        success, image = vidObj.read()
        frames.append(image)

    frame_usage = [0 for i in range(len(frames))]

    # Divide Original video into frames
    vidObj_or = cv2.VideoCapture(videoname) 
    success = True
    frames_or = []
    while success: 
        success, image_or = vidObj_or.read()
        frames_or.append(image_or)

    print(frames, frames_or)
    
    # Set Limit
    frame_size = (len(frames[0]), len(frames[0][0]))
    print(frame_size)
    frame_usage_limit = (frame_size[0] * frame_size[1] * 3) * 0.3

    data_index = 0
    data = []
    current = ""

    # Build LFSR to be used to choose frame
    frame_lfsr = LFSR(initstate = [1, 1, 1, 1, 1, 1, 1, 1], fpoly=poly, counter_start_zero=True)
    frame_index = 0

    while data_index < data_len-1:

        print("frame loop entered")

        # Choose Frame using LFSR
        frame_index = (frame_index + binaryToDecimal(frame_lfsr.state)) % len(frames)
        frame_lfsr.next()

        # If frame is already used, keep choosing next frame until unused frame is found.
        count = 0
        while frame_usage[frame_index] != 0:
            frame_index = (frame_index + binaryToDecimal(frame_lfsr.state)) % len(frames)
            frame_lfsr.next()
            count += 1
            if count > 100:
                print("frames exceeded")
                return
            
        print("frame selected", frame_index)

        # Build chaotic map and derive some available pixels for the frame
        map = HennonMapBinary(frame_size, hennonKey)

        print("checkpoint 1")

        availablePixels = []

        for i in range(frame_size[0]):
            for j in range(0,frame_size[1]):
                if map[i][j] == 1:
                    availablePixels.append([i,j,0])
        
        print("checkpoint 2")

        # Build LFSR for pixel choice
        pixel_lfsr = LFSR(initstate = [1, 1, 1, 1, 1, 1, 1, 1], fpoly=poly, counter_start_zero=True)
        pixel_index = 0

        print("map created")

        while data_index < data_len-1:

            print("pixel loop entered", data_index)

            # Choose pixel using LFSR from Available pixels
            pixel_index = (pixel_index + binaryToDecimal(pixel_lfsr.state)) % len(availablePixels)
            pixel_lfsr.next()

            # If pixel is already used 3 times, keep choosing next pixel until unused pixel is found.
            count = 0
            while availablePixels[pixel_index][2] > 2:
                pixel_index = (pixel_index + binaryToDecimal(pixel_lfsr.state)) % len(availablePixels)
                pixel_lfsr.next()
                count += 1
                if count > 100:
                    print("pixels exceeded")
                    return
            
            print("pixel selected", pixel_index)

            '''
            EACH PIXEL CAN BE USED AT MAX 3 TIMES. WE KEEP COUNT OF NUMBER OF USES AS USAGESTATE.

            IF USAGESTATE IS 0: XOR 2 DATA BITS WITH 2 LSB OF RED VALUE
            IF USAGESTATE IS 1: XOR 2 DATA BITS WITH 2 LSB OF BLUE VALUE
            IF USAGESTATE IS 2: XOR 2 DATA BITS WITH 2 LSB OF GREEN VALUE
            '''

            # Extract row, column and usagestate values.
            row = availablePixels[pixel_index][0]
            col = availablePixels[pixel_index][1]
            usageState = availablePixels[pixel_index][2]

            # Update with Red Value
            if usageState == 0:
                value = list(convertToBinary(frames[frame_index][row,col][0]))
                value_or = list(convertToBinary(frames_or[frame_index][row,col][0]))
                
                temp1 = str(int(value[-2]) ^ int(value_or[-2]))
                temp2 = str(int(value[-1]) ^ int(value_or[-1]))

            # Extract from Blue Value
            elif usageState == 1:
                value = list(convertToBinary(frames[frame_index][row,col][1]))
                value_or = list(convertToBinary(frames_or[frame_index][row,col][1]))
                
                temp1 = str(int(value[-2]) ^ int(value_or[-2]))
                temp2 = str(int(value[-1]) ^ int(value_or[-1]))
            
            # Extract from Green Value
            else:
                value = list(convertToBinary(frames[frame_index][row,col][2]))
                value_or = list(convertToBinary(frames_or[frame_index][row,col][2]))

                temp1 = str(int(value[-2]) ^ int(value_or[-2]))
                temp2 = str(int(value[-1]) ^ int(value_or[-1]))
            
            current = current + temp1 + temp2
            if len(current) == 8:
                data.append(current)
                current = ""
            
            data_index += 2
            availablePixels[pixel_index][2] += 1
            frame_usage[frame_index] += 1
            if frame_usage[frame_index] >= frame_usage_limit:
                break
        
    data_found = "".join(chr(int(c, 2)) for c in data)
    print(data_found)

    # Store Output in File
    filehandler = open("text\output.txt","w")
    filehandler.writelines(data_found)
    filehandler.close()

    return data_found

In [70]:
import json

def main():
    info = open('config.json')
    json_data = json.load(info)
    original_image = json_data["source"]["video_filename"]
    hennon_key = json_data["key"]["hennon_key"]
    lfsr_poly = json_data["key"]["lfsr_polynomial"]
    
    decode(original_image, hennon_key, lfsr_poly, 64)

In [73]:
main()

[array([[[202, 207, 207],
        [202, 207, 207],
        [202, 207, 207],
        ...,
        [201, 197, 200],
        [201, 197, 200],
        [201, 197, 200]],

       [[202, 207, 207],
        [202, 207, 207],
        [202, 207, 207],
        ...,
        [201, 197, 200],
        [201, 197, 200],
        [201, 197, 200]],

       [[202, 207, 207],
        [202, 207, 207],
        [202, 207, 207],
        ...,
        [201, 197, 200],
        [201, 197, 200],
        [201, 197, 200]],

       ...,

       [[135, 140, 140],
        [135, 140, 140],
        [135, 140, 140],
        ...,
        [133, 139, 134],
        [133, 139, 134],
        [133, 139, 134]],

       [[135, 140, 140],
        [135, 140, 140],
        [135, 140, 140],
        ...,
        [133, 139, 134],
        [133, 139, 134],
        [133, 139, 134]],

       [[135, 140, 140],
        [135, 140, 140],
        [135, 140, 140],
        ...,
        [133, 139, 134],
        [133, 139, 134],
        [133, 139, 134]

UnicodeEncodeError: 'charmap' codec can't encode character '\x89' in position 0: character maps to <undefined>